In [1]:
import $ivy.`org.apache.spark::spark-sql:3.2.0`
import $ivy.`sh.almond::almond-spark:0.10.1`

import org.apache.spark._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}

val spark = SparkSession
      .builder()
      .config("spark.sql.shuffle.partitions", 4)
      .master("local[4]")
      .getOrCreate()

import spark.implicits._

Logger.getRootLogger().setLevel(Level.ERROR)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
22/05/05 10:04:37 INFO SparkContext: Running Spark version 3.2.0
22/05/05 10:04:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/05 10:04:37 INFO ResourceUtils: ==============================================================
22/05/05 10:04:37 INFO ResourceUtils: No custom resources configured for spark.driver.
22/05/05 10:04:37 INFO ResourceUtils: ==============================================================
22/05/05 10:04:37 INFO SparkContext: Submitted application: 08281ef8-428a-4ded-a90e-47881a889212
22/05/05 10:04:37 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
22/05/0

import $ivy.$                                  

import $ivy.$                               


import org.apache.spark._

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._


import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}


spark: SparkSession = org.apache.spark.sql.SparkSession@2d4314f9
import spark.implicits._



In [2]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.0`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// if you want to have the plots available without an internet connection:
//init(offline=true)

// restrict the output height to avoid scrolling in output cells
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

import $ivy.$                                      

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// if you want to have the plots available without an internet connection:
//init(offline=true)

// restrict the output height to avoid scrolling in output cells


In [3]:
import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.expressions.Window

In [4]:
val races = spark.read.parquet("../../../data/parquetnopart/races.parquet")
    .select("raceId", "year")
    .where(col("year") === 2021)

races: Dataset[Row] = [raceId: string, year: string]

In [5]:
val drivers = spark.read.parquet("../../../data/parquetnopart/drivers.parquet")

drivers: DataFrame = [driverId: string, driverRef: string ... 7 more fields]

In [6]:
val driverWindow = Window.partitionBy("driverId")
val seasonWindow = Window.partitionBy("year")
val driverRaceWindow = Window.partitionBy("driverId", "raceId")
val raceDriverLapWindow = Window.partitionBy("driverId", "raceId").orderBy("lap")

driverWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@3c4040bb
seasonWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@1dd4ca2c
driverRaceWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@605490fa
raceDriverLapWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@107fc405

In [7]:
val driverStats = spark.read.parquet("../../../data/parquetnopart/lap_times.parquet")

    .withColumn("position", col("position").cast(IntegerType)) 
    .withColumn("lap", col("lap").cast(IntegerType)) 
    .join(races, "raceId")

    .withColumn("positionNextLap", lead(col("position"), 1).over(raceDriverLapWindow))
    .withColumn("positionsGainedLap", when(col("positionNextLap") < col("position") , abs(col("position") - col("positionNextLap"))).otherwise(0))
    .withColumn("positionsLostLap", when(col("positionNextLap") > col("position"), abs(col("position") - col("positionNextLap"))).otherwise(0))
    .withColumn("positionsGained", sum(col("positionsGainedLap")).over(driverRaceWindow))
    .withColumn("positionsLost", sum(col("positionsLostLap")).over(driverRaceWindow))
    .withColumn("lapLeader", when(col("position") === 1, 1).otherwise(0))
    .withColumn("lapsLed", sum(col("lapLeader")).over(driverWindow))
    .withColumn("totalLaps", sum(col("lapLeader")).over(seasonWindow))
    .withColumn("percLapsLed", round(col("lapsLed") / col("totalLaps"), 2))
    .select("raceId", "driverId", "positionsGained", "positionsLost", "lapsLed", "percLapsLed")
    .dropDuplicates()

driverStats: Dataset[Row] = [raceId: string, driverId: string ... 4 more fields]

In [8]:
val results = spark.read.parquet("../../../data/parquetnopart/results.parquet")

    .withColumn("position", col("position").cast(IntegerType))    
    .withColumn("grid", col("grid").cast(IntegerType))    
    .withColumn("points", col("points").cast(IntegerType))

    .join(races, "raceId")
    .join(driverStats, Seq("raceId", "driverId"), "left")
    .join(drivers, "driverId")

    .withColumn("podium", when(col("position") === 1 || col("position") === 2 ||col("position") === 3, lit(1)).otherwise(lit(0)))
    .withColumn("averagePoints", round(avg(col("points")).over(driverWindow), 2))
    .withColumn("maxAvgPoints", max(col("averagePoints")).over(seasonWindow))

    .select(
        col("code"),
        sum(col("points")).over(driverWindow).as("champPoints"),
        col("averagePoints"),
        round(col("averagePoints") / col("maxAvgPoints"),2).as("pointPercent"),
        sum(col("podium")).over(driverWindow).as("totalPodiums"),        
        round(avg(col("position")).over(driverWindow), 2).as("avgPosition"),
        round(sum(col("podium")).over(driverWindow) / count(col("podium")).over(driverWindow), 2).as("podiumPercent"),
        round(avg(col("position") - col("grid")).over(driverWindow), 2).as("positionDelta"),
        round(avg(col("positionsLost")).over(driverWindow), 2).as("avgPositionsLost"),
        round(avg(col("positionsGained")).over(driverWindow), 2).as("avgPositionsWon"),
        sum(col("positionsLost")).over(driverWindow).as("totalPositionsLost"),
        sum(col("positionsGained")).over(driverWindow).as("totalPositionsWon"),
        col("lapsLed"),
        col("percLapsLed")
    )

    .na.fill(0)
    .dropDuplicates(Seq("code"))

results: Dataset[Row] = [code: string, champPoints: bigint ... 12 more fields]

In [8]:
// results.show(false)

In [8]:
// val driverCodes = results
//     .sort("avgPosition")
//     .select("code")
//     .as[String]
//     .collect()
//     .toList

// val positionDelta = results
//     .sort("positionDelta")
//     .select("positionDelta")
//     .as[Double]
//     .collect()
//     .toList

// val data = Seq(
//   Bar(
//       driverCodes,  
//       positionDelta
//   )
// )

// val layout = Layout( 
//   barmode = BarMode.Group
// )

// plot(data, layout)


In [9]:
def run[A](code: => A): Long = {
    val start = System.currentTimeMillis()
    val res = code
    System.currentTimeMillis() - start
}

def average(list: List[Long]):Long = list.sum / list.size.toLong

def timeTest(f: => Long): Long = {
    val list = (1 to 1).map(_ => f).toList
    average(list)
}

val res = timeTest(run(results.collect))

defined function run
defined function average
defined function timeTest
res: Long = 2451L